In [ ]:
# for GPU 
"""
import os

os.environ['PIP_CACHE_DIR'] = '/workspace/workspace/cache'

os.environ['TRANSFORMERS_CACHE'] = '/workspace/workspace/cache'
os.environ['TORCH_CACHE'] = '/workspace/workspace/cache'
os.environ['HF_CACHE'] = '/workspace/workspace/cache'

import kagglehub
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import fasttext.util as fasttext_util
import fasttext
from sklearn.metrics import f1_score
from collections import Counter
from torch.utils.data import DataLoader
import torch.nn as nn
import torch

import fasttext.util
fasttext.util.download_model('en', if_exists='ignore', path='/workspace/workspace/cache')
"""


In [14]:
import os
import kagglehub
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import fasttext.util as fasttext_util
import fasttext 
from sklearn.metrics import f1_score
from collections import Counter
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import tqdm

In [3]:
# specify custom functions --------------------------------------------------------------
def custom_tokenizer(text):
    tokenized_text = nlp(text)
    return [tok.text for tok in tokenized_text]

def embedding_mapping_fasttext(vocabulary, pre_trained_embeddings):
    vocab_size = len(vocabulary)
    embedding_dim = pre_trained_embeddings.get_dimension()
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for idx, word in enumerate(vocabulary):
        embedding_matrix[idx] = pre_trained_embeddings.get_word_vector(word)
    return embedding_matrix

# download pretrained embeddings --------------------------------------------------------
fasttext_util.download_model('en', if_exists='ignore')

# download spacy model for tokenization -------------------------------------------------
spacy.cli.download("en_core_web_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 39.7 MB/s eta 0:00:00 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Load rumours dataset

In [ ]:
"""
os.chdir("/Users/henrybaker/Documents/repositories/nlp/nlp_research_note")
print("Working Directory:", os.getcwd())

path_rumour = kagglehub.dataset_download("syntheticprogrammer/rumor-detection-acl-2017")
print("Path to rumour dataset files:", path_rumour)

path_climate = kagglehub.dataset_download("die9origephit/climate-change-tweets")
print("Path to dataset files:", path_climate)

# List all files in the directory
files_rumour = os.listdir(path_rumour)
files_climate = os.listdir(path_climate)

# Print the list of files
print("Files in rumour dataset directory:", files_rumour)
print("Files in climate dataset directory:", files_climate)
"""

In [ ]:
""" 
combined_data = []

for numb in [15, 16]:
    path_numb = files_rumour[numb - 15]  # 'twitter15' or 'twitter16' folder path
    label_file_path_numb = os.path.join(path_rumour, path_numb, 'label.txt')
    tweets_file_path_numb = os.path.join(path_rumour, path_numb, 'source_tweets.txt')

    # Read label.txt
    label_dict = {}
    with open(label_file_path_numb, 'r') as file:
        for line in file:
            label, tweet_id = line.strip().split(':')
            label_dict[tweet_id] = label

    # Read source_tweets.txt
    tweets_dict = {}
    with open(tweets_file_path_numb, 'r') as file:
        for line in file:
            tweet_id, tweet_content = line.strip().split('\t', 1)
            tweets_dict[tweet_id] = tweet_content

    # Combine labels with tweets
    for tweet_id, tweet_content in tweets_dict.items():
        if tweet_id in label_dict:
            combined_data.append((label_dict[tweet_id], tweet_content))

    print(f"twitter_{numb}:") 

    for label, tweet in combined_data[:5]:
        print(f"    Label: {label}, Tweet: {tweet}")
"""

crop URL part
chosen specifically because it is twitter

In [ ]:
""" 
print(f"Total number of entries in combined data: {len(combined_data)} \n")

unique_labels = set(label for label, tweet in combined_data)

print("Unique labels in combined data:")
for label in unique_labels:
    print(f"   ", label)
"""

In [ ]:
""" 
df_combined = pd.DataFrame(combined_data, columns=["Label", "Tweet"])

print(df_combined.shape)
df_combined.head()
"""

In [ ]:
""" 
grouped = df_combined.groupby("Label")

# 5 random examples for each label
for label, group in grouped:
    print(f"\nLabel: {label}")
    sample = group.sample(n=5, random_state=42)  
    for _, row in sample.iterrows():
        print(f"   Tweet: {row['Tweet']}")
"""

--- 
# Hugging Face misinfo dataset

In [6]:
# Hugging Face
ds = load_dataset("roupenminassian/twitter-misinformation")
hf_cache_dir = os.getenv("HF_DATASETS_CACHE", "~/.cache/huggingface/datasets")
print(hf_cache_dir)

~/.cache/huggingface/datasets


need to balance the dataset?

In [7]:
print(ds.shape)

print(ds['train'].column_names)
print(ds['train'][:5]) 

{'train': (92394, 4), 'test': (10267, 4)}
['Unnamed: 0.1', 'Unnamed: 0', 'text', 'label']
{'Unnamed: 0.1': [34366, 41656, 26726, 81585, 4016], 'Unnamed: 0': [34366, 41656, 26726, 81585, 4016], 'text': ["Local Charlotte, NC news station WSOCTV is reporting that sources tell them dash cameras captured Keith Scott getting out of car and coming towards officers with a gun in his hand:#BREAKING: Sources tell Channel 9 dash camera video shows #KeithScott getting out car, coming toward officers with gun in his hand pic.twitter.com/GGuM2Ow3wk  WSOCTV (@wsoctv) September 21, 2016For a second night, protests over a deadly officer-involved shooting in Charlotte, North Carolina, turned violent, with police firing tear gas and demonstrators throwing objects and trying to damage vehicles.Keith Lamont Scott, a father of seven, was killed by police in an apartment complex parking lot Tuesday as officers looked for another man named in a warrant they were trying to serve. The shooting set off a long ni

In [8]:
ds_cloned = ds.copy()

# DATA PARTITIONING =====================================================================

ds_cloned['train'] = ds_cloned['train'].remove_columns(['Unnamed: 0', 'Unnamed: 0.1'])
ds_cloned['test'] = ds_cloned['test'].remove_columns(['Unnamed: 0', 'Unnamed: 0.1'])

df_misinfo_train = pd.DataFrame(ds_cloned['train'], columns=["text", "label"])
df_misinfo_test = pd.DataFrame(ds_cloned['test'], columns=["text", "label"])


print(f"Train shape {df_misinfo_train.shape} \n")
print("Training positive vs negative examples: \n", df_misinfo_train.value_counts("label")/df_misinfo_train.shape[0])
print("\nTesting positive vs negative examples: \n",df_misinfo_test.value_counts("label")/df_misinfo_test.shape[0])

df_misinfo_train.head()

Train shape (92394, 2) 

Training positive vs negative examples: 
 label
0    0.652737
1    0.347263
Name: count, dtype: float64

Testing positive vs negative examples: 
 label
0    0.659686
1    0.340314
Name: count, dtype: float64


,text,label
0,"Local Charlotte, NC news station WSOCTV is rep...",1
1,The tsunami has started President Obama s Keny...,1
2,The only reality show Donald Trump should have...,1
3,"No Food, No FEMA: Hurricane Michael’s Survivor...",0
4,WASHINGTON (Reuters) - Here are some of the hi...,0


In [ ]:
# balance train split -------------------------------------------------------------------
'''
balancer = RandomUnderSampler(random_state=42, sampling_strategy = 'majority')
df_misinfo_train_balanced = pd.concat(balancer.fit_resample(X = df_misinfo_train.iloc[:,[0]],
                                                           y = df_misinfo_train.iloc[:,[1]]),
                                     axis=1).sample(frac = 1).reset_index(drop=True)
df_misinfo_train_balanced.value_counts("y")/df_misinfo_train_balanced.shape[0]
df_misinfo_train_balanced.shape[0]
df_misinfo_train.shape[0]
'''

In [17]:
'''
# specify custom functions --------------------------------------------------------------
def custom_tokenizer(text):
    tokenized_text = nlp(text)
    return [tok.text for tok in tokenized_text]

def embedding_mapping_fasttext(vocabulary, pre_trained_embeddings):
    vocab_size = len(vocabulary)
    embedding_dim = pre_trained_embeddings.get_dimension()
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for idx, word in enumerate(vocabulary):
        embedding_matrix[idx] = pre_trained_embeddings.get_word_vector(word)
    return embedding_matrix

# download pretrained embeddings --------------------------------------------------------
fasttext_util.download_model('en', if_exists='ignore')

# download spacy model for tokenization -------------------------------------------------
spacy.cli.download("en_core_web_sm")
'''

# TOKENIZATION ==========================================================================
# tokenize texts ------------------------------------------------------------------------
print("Loading spaCy model...")
nlp = spacy.load("en_core_web_sm", disable=["tok2vec", "tagger", "parser", "ner", "lemmatizer", "attribute_ruler"])

# Fit tokenizer with progress bar
misinfo_tokenizer = CountVectorizer(analyzer="word",
                                   tokenizer=custom_tokenizer,
                                   lowercase=False,
                                   min_df=3)

print("Fitting Tokenizer...")
misinfo_tokenizer.fit(df_misinfo_train["text"])

misinfo_tokenizer_analyzer = misinfo_tokenizer.build_analyzer()


print("Tokenizing Train Data...")
misinfo_train_tokens = df_misinfo_train["text"].map(misinfo_tokenizer_analyzer)

print("Tokenizing Test Data...")
misinfo_test_tokens = df_misinfo_test["text"].apply(misinfo_tokenizer_analyzer, trained_tokenizer=misinfo_tokenizer)

def custom_analyzer(text, trained_tokenizer):
    tokens = custom_tokenizer(text)
    vocab = trained_tokenizer.vocabulary_
    return [token if token in vocab else "<unk>" for token in tokens]

misinfo_test_tokens = df_misinfo_test["text"].apply(custom_analyzer, trained_tokenizer=misinfo_tokenizer)

Loading spaCy model...
Fitting Tokenizer...
Tokenizing Train Data...


KeyboardInterrupt: 

In [ ]:
# STEP 1: INPUT PIPELINE ================================================================

# vocabulary indexing -------------------------------------------------------------------
def vocab_mapping(tokenized_text):
    token_counts = Counter()
    for text in tokenized_text:
        token_counts.update(text)
    special_tokens = ["<pad>", "<unk>"]
    vocab_tokens = special_tokens + [token for token, freq in token_counts.most_common()]
    vocab = {token: idx for idx, token in enumerate(vocab_tokens)}
    return vocab

vocab_idx = vocab_mapping(tokenized_text=reddit_train_tokens)


---

In [ ]:
# reading climate df
input_path_climate = "/Users/henrybaker/.cache/kagglehub/datasets/die9origephit/climate-change-tweets/versions/1/Climate change_2022-1-17_2022-7-19.csv" 

output_path_climate = "/Users/henrybaker/Documents/repositories/NLP/nlp_project/data/climate-change-tweets.csv"

df_climate = pd.read_csv(input_path_climate)
print(f"Loading dataset from '{input_path_climate}'...")
df_climate.head()